# Data Merging Basics

## One-to-One Relationships

### Merging Census data with Chicago local gov data

The city of Chicago is divided into fifty local neighborhoods called wards. We have a table with data about the local government offices in each ward. 

- In this example, we will merge the local government data with census data about the population of each ward.

In [2]:
import pandas as pd

In [9]:
wards = pd.read_pickle("../data/pandas_datasets/pandas-joins/ward.p")
wards.head(10)

,ward,alderman,address,zip
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647
1,2,Brian Hopkins,1400 NORTH ASHLAND AVENUE,60622
2,3,Pat Dowell,5046 SOUTH STATE STREET,60609
3,4,William D. Burns,"435 EAST 35TH STREET, 1ST FLOOR",60616
4,5,Leslie A. Hairston,2325 EAST 71ST STREET,60649
5,6,Roderick T. Sawyer,8001 S. MARTIN LUTHER KING DRIVE,60619
6,7,Gregory I. Mitchell,2249 EAST 95TH STREET,60617
7,8,Michelle A. Harris,8539 SOUTH COTTAGE GROVE AVENUE,60619
8,9,Anthony A. Beale,34 EAST 112TH PLACE,60628
9,10,Susan Sadlowski Garza,10500 SOUTH EWING AVENUE,60617


In [10]:
wards.shape

(50, 4)

We have information about the local government of each ward, such as the government office address. This table has 50 rows and 4 columns

In [12]:
census = pd.read_pickle("../data/pandas_datasets/pandas-joins/census.p")
census.head(10)

,ward,pop_2000,pop_2010,change,address,zip
0,1,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622
2,3,40385,53039,31%,17 EAST 38TH STREET,60653
3,4,51953,54589,5%,31ST ST HARBOR BUILDING LAKEFRONT TRAIL,60653
4,5,55302,51455,-7%,JACKSON PARK LAGOON SOUTH CORNELL DRIVE,60637
5,6,54989,52341,-5%,150 WEST 74TH STREET,60636
6,7,54593,51581,-6%,8549 SOUTH OGLESBY AVENUE,60617
7,8,54039,51687,-4%,1346-1352 EAST 75TH STREET,60649
8,9,52008,51519,-1%,11039-11059 SOUTH WENTWORTH AVENUE,60628
9,10,56613,51535,-9%,10534 SOUTH AVENUE F,46394


In [13]:
census.shape

(50, 6)

The census table contains the population of each ward in 2000 and 2010, and that change as a percentage. Additionally, it includes the address for the center of each ward. This table has 50 rows and 6 columns.

The two tables are related by their ward column. We can merge them together, matching the ward number from each row of the wards table to the ward numbers from the census table. For example, the second ward in the wards table with Alderman Brian Hopkins would be matched with row 2 of the census table where the population in 2000 was 54,361.

***Inner Join***

- The pandas package has an excellent DataFrame method for performing this type of merge called merge. The merge method takes the first DataFrame, wards, and merges it with the second DataFrame, census. We use the on argument to tell the method that we want to merge the two DataFrames on the ward column. 

- Since we listed the wards table first, its columns will appear first in the output, followed by the columns from the census table. In this example, the merge returns a DataFrame with 50 rows and 9 columns, where the returned rows have matching values for the ward column in both tables. This is called an **inner join**.

<img src="../img/inner-join.jpg" width="400" height="300">

In [16]:
wards_census = wards.merge(census, on="ward")
wards_census.head(5)

,ward,alderman,address_x,zip_x,pop_2000,pop_2010,change,address_y,zip_y
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,Brian Hopkins,1400 NORTH ASHLAND AVENUE,60622,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622
2,3,Pat Dowell,5046 SOUTH STATE STREET,60609,40385,53039,31%,17 EAST 38TH STREET,60653
3,4,William D. Burns,"435 EAST 35TH STREET, 1ST FLOOR",60616,51953,54589,5%,31ST ST HARBOR BUILDING LAKEFRONT TRAIL,60653
4,5,Leslie A. Hairston,2325 EAST 71ST STREET,60649,55302,51455,-7%,JACKSON PARK LAGOON SOUTH CORNELL DRIVE,60637


An inner join will only return rows that have matching values in both tables.

In [17]:
print(wards_census.columns)

Index(['ward', 'alderman', 'address_x', 'zip_x', 'pop_2000', 'pop_2010',
       'change', 'address_y', 'zip_y'],
      dtype='object')


The merged table has columns with suffixes of underscore x or y. This is because both the wards and census tables contained address and zip columns. To avoid multiple columns with the same name, they are automatically given a suffix by the merge method.

In [18]:
wards_census = wards.merge(census, on="ward", suffixes=("_ward", "_cen"))
wards_census.head(5)

,ward,alderman,address_ward,zip_ward,pop_2000,pop_2010,change,address_cen,zip_cen
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,Brian Hopkins,1400 NORTH ASHLAND AVENUE,60622,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622
2,3,Pat Dowell,5046 SOUTH STATE STREET,60609,40385,53039,31%,17 EAST 38TH STREET,60653
3,4,William D. Burns,"435 EAST 35TH STREET, 1ST FLOOR",60616,51953,54589,5%,31ST ST HARBOR BUILDING LAKEFRONT TRAIL,60653
4,5,Leslie A. Hairston,2325 EAST 71ST STREET,60649,55302,51455,-7%,JACKSON PARK LAGOON SOUTH CORNELL DRIVE,60637


We can use the suffix argument of the merge method to control this behavior. We provide a tuple where all of the overlapping columns in the left table are given the suffix '_ward', and those of the right table will be given the suffix '_cen'. This makes it easier for us to tell the difference between the columns.

### Taxicab owners and vehicle info data

Chicago provides a list of taxicab owners and vehicles licensed to operate within the city, for public safety. 

- We'll merge two tables together. One table is called taxi_owners, with info about the taxi cab company owners, and one is called taxi_veh, with info about each taxi cab vehicle

#### 1. Let's figure out what the most popular types of fuel used in Chicago taxis are.

In [19]:
taxi_owners = pd.read_pickle("../data/pandas_datasets/pandas-joins/taxi_owners.p")
taxi_vehicles = pd.read_pickle("../data/pandas_datasets/pandas-joins/taxi_vehicles.p")

In [20]:
taxi_owners.head(5)

,rid,vid,owner,address,zip
0,T6285,6285,AGEAN TAXI LLC,4536 N. ELSTON AVE.,60630
1,T4862,4862,MANGIB CORP.,5717 N. WASHTENAW AVE.,60659
2,T1495,1495,"FUNRIDE, INC.",3351 W. ADDISON ST.,60618
3,T4231,4231,ALQUSH CORP.,6611 N. CAMPBELL AVE.,60645
4,T5971,5971,EUNIFFORD INC.,3351 W. ADDISON ST.,60618


In [21]:
taxi_owners.shape

(3519, 5)

In [22]:
taxi_vehicles.head(5)

,vid,make,model,year,fuel_type,owner
0,2767,TOYOTA,CAMRY,2013,HYBRID,SEYED M. BADRI
1,1411,TOYOTA,RAV4,2017,HYBRID,DESZY CORP.
2,6500,NISSAN,SENTRA,2019,GASOLINE,AGAPH CAB CORP
3,2746,TOYOTA,CAMRY,2013,HYBRID,"MIDWEST CAB CO, INC"
4,5922,TOYOTA,CAMRY,2013,HYBRID,SUMETTI CAB CO


In [23]:
taxi_vehicles.shape

(3519, 6)

In [24]:
# Merge the taxi_owners and taxi_veh tables setting a suffix
taxi_own_veh = taxi_owners.merge(taxi_vehicles, on='vid', suffixes=('_own','_veh'))

# Print the value_counts to find the most popular fuel_type
print(taxi_own_veh['fuel_type'].value_counts())

fuel_type
HYBRID                    2792
GASOLINE                   611
FLEX FUEL                   89
COMPRESSED NATURAL GAS      27
Name: count, dtype: int64


## One-to-Many relationships

Let's explore one-to-many merges by merging a table of business owners, called `biz_owners`, to the `licenses` table and find out what is the most common business owner title. (i.e., secretary, CEO, or vice president).

In [25]:
licenses = pd.read_pickle("../data/pandas_datasets/pandas-joins/licenses.p")
biz_owners = pd.read_pickle("../data/pandas_datasets/pandas-joins/business_owners.p")

In [26]:
# Merge the licenses and biz_owners table on account
licenses_owners = licenses.merge(biz_owners, on="account")

# Group the results by title then count the number of accounts
counted_df = licenses_owners.groupby("title").agg({'account':'count'})

# Sort the counted_df in descending order
sorted_df = counted_df.sort_values(by="account", ascending=False)

# Use .head() method to print the first few rows of sorted_df
print(sorted_df.head())

                 account
title                   
PRESIDENT           6259
SECRETARY           5205
SOLE PROPRIETOR     1658
OTHER               1200
VICE PRESIDENT       970


## Merging Multiple Data Frames

Determine the total number of rides provided to passengers passing through the Wilson station `(station_name == 'Wilson')` on weekdays `(day_type == 'Weekday')` in July `(month == 7)` using Chicago's public transportation data. The data is available in three separate tables that need to be merged to obtain the required insights. The `cal`, `ridership`, and `stations` DataFrames have been loaded, and their relationships are shown in the diagram below.

<img src="../img/cta_ridership.jpg" width="400">

In [27]:
ridership = pd.read_pickle("../data/pandas_datasets/pandas-joins/cta_ridership.p")
ridership

,station_id,year,month,day,rides
0,40010,2019,1,1,576
1,40010,2019,1,2,1457
2,40010,2019,1,3,1543
3,40010,2019,1,4,1621
4,40010,2019,1,5,719
...,...,...,...,...,...
3280,41660,2019,12,27,13898
3281,41660,2019,12,28,9485
3282,41660,2019,12,29,7581
3283,41660,2019,12,30,15332


In [28]:
cal = pd.read_pickle("../data/pandas_datasets/pandas-joins/cta_calendar.p")
cal

,year,month,day,day_type
0,2019,1,1,Sunday/Holiday
1,2019,1,2,Weekday
2,2019,1,3,Weekday
3,2019,1,4,Weekday
4,2019,1,5,Saturday
...,...,...,...,...
360,2019,12,27,Weekday
361,2019,12,28,Saturday
362,2019,12,29,Sunday/Holiday
363,2019,12,30,Weekday


In [29]:
stations = pd.read_pickle("../data/pandas_datasets/pandas-joins/stations.p")
stations

,station_id,station_name,location
0,40010,Austin-Forest Park,"(41.870851, -87.776812)"
1,40020,Harlem-Lake,"(41.886848, -87.803176)"
2,40030,Pulaski-Lake,"(41.885412, -87.725404)"
3,40040,Quincy/Wells,"(41.878723, -87.63374)"
4,40050,Davis,"(42.04771, -87.683543)"
...,...,...,...
139,41660,Lake/State,"(41.884809, -87.627813)"
140,41670,Conservatory,"(41.884904, -87.716523)"
141,41680,Oakton-Skokie,"(42.02624348, -87.74722084)"
142,41690,Cermak-McCormick Place,"(41.853115, -87.626402)"


In [30]:
# Total riders in a month
# Merge the ridership, cal, and stations tables
ridership_cal_stations = ridership.merge(cal, on=['year','month','day']) \
							.merge(stations, on='station_id')

In [31]:
# Create a filter to filter ridership_cal_stations
filter_criteria = ((ridership_cal_stations['month'] == 7) 
                   & (ridership_cal_stations['day_type'] == 'Weekday') 
                   & (ridership_cal_stations['station_name'] == 'Wilson'))

In [32]:
# Use .loc and the filter to select for rides
print(ridership_cal_stations.loc[filter_criteria, 'rides'].sum())

140005


Wilson station had 140,005 riders during weekdays in July.

In [36]:
zip_demo = pd.read_pickle("../data/pandas_datasets/pandas-joins/zip_demo.p")
zip_demo

,zip,income
0,60630,70122
1,60640,50488
2,60622,87143
3,60614,100116
4,60608,41226
...,...,...
61,53045,100438
62,60076,74964
63,60126,103454
64,60458,47411


In [37]:
# Merge licenses and zip_demo, on zip; and merge the wards on ward
licenses_zip_ward = licenses.merge(zip_demo, on="zip") \
							.merge(wards, on="ward")


# Print the results by alderman and show median income
print(licenses_zip_ward.groupby("alderman").agg({'income':'median'}))

                             income
alderman                           
Ameya Pawar                 66246.0
Anthony A. Beale            38206.0
Anthony V. Napolitano       82226.0
Ariel E. Reyboras           41307.0
Brendan Reilly             110215.0
Brian Hopkins               87143.0
Carlos Ramirez-Rosa         66246.0
Carrie M. Austin            38206.0
Chris Taliaferro            55566.0
Daniel "Danny" Solis        41226.0
David H. Moore              33304.0
Deborah Mell                66246.0
Debra L. Silverstein        50554.0
Derrick G. Curtis           65770.0
Edward M. Burke             42335.0
Emma M. Mitts               36283.0
George Cardenas             33959.0
Gilbert Villegas            41307.0
Gregory I. Mitchell         24941.0
Harry Osterman              45442.0
Howard B. Brookins, Jr.     33304.0
James Cappleman             79565.0
Jason C. Ervin              41226.0
Joe Moore                   39163.0
John S. Arena               70122.0
Leslie A. Hairston          

We see that only a few aldermen represent businesses in areas where the median income is greater than $62,000, which is the median income for the state of Illinois.

## One-to-many merge with multiple tables

Identifying an optimal location in Chicago for a goat-based lawn mowing business. The ideal location should have ample space and a low density of businesses and residents to minimize potential complaints. To determine the best area, three tables will be merged to analyze space availability and population distribution.

In [39]:
land_use = pd.read_pickle("../data/pandas_datasets/pandas-joins/land_use.p")
land_use.head(2)

,ward,residential,commercial,industrial,vacant,other
0,1,41,9,2,2,46
1,2,31,11,6,2,50


In [40]:
licenses.head(2)

,account,ward,aid,business,address,zip
0,307071,3,743,REGGIE'S BAR & GRILL,2105 S STATE ST,60616
1,10,10,829,HONEYBEERS,13200 S HOUSTON AVE,60633


In [41]:
census.head(2)

,ward,pop_2000,pop_2010,change,address,zip
0,1,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622


In [43]:

# Merge land_use and census and merge result with licenses including suffixes
land_cen_lic = land_use.merge(census, on='ward') \
                    .merge(licenses, on='ward', suffixes=('_cen','_lic'))

# Group by ward, pop_2010, and vacant, then count the # of accounts
pop_vac_lic = land_cen_lic.groupby(['ward','pop_2010','vacant'], 
                                   as_index=False).agg({'account':'count'})

In [44]:
land_cen_lic.head(5)

,ward,residential,commercial,industrial,vacant,other,pop_2000,pop_2010,change,address_cen,zip_cen,account,aid,business,address_lic,zip_lic
0,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,12024,NaN,DIGILOG ELECTRONICS,1038 N ASHLAND AVE,60622
1,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,14446,743,EMPTY BOTTLE INC,1035 N WESTERN AVE 1ST,60622
2,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,14624,775,LITTLE MEL'S HOT DOG,2205 N CALIFORNIA AVE,60647
3,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,14987,NaN,MR. BROWN'S LOUNGE,2301 W CHICAGO AVE 1ST,60622
4,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,15642,814,Beat Kitchen,2000-2100 W DIVISION ST,60622


In [45]:
pop_vac_lic.head(5)

,ward,pop_2010,vacant,account
0,1,56149,2,253
1,10,51535,14,130
2,11,51497,5,201
3,12,52235,4,255
4,13,53722,1,101


In [46]:
# Sort pop_vac_lic and print the results
sorted_pop_vac_lic = pop_vac_lic.sort_values(["vacant","account","pop_2010"], 
                                             ascending=(False, True, True))

# Print the top few rows of sorted_pop_vac_lic
print(sorted_pop_vac_lic.head())

   ward  pop_2010  vacant  account
47    7     51581      19       80
12   20     52372      15      123
1    10     51535      14      130
16   24     54909      13       98
7    16     51954      13      156


The 7th ward would be a good place to build the goat farm!

# Merging Tables With Different Join Types

# Advanced Merging and Concatenating

# Merging Ordered and Time-Series Data